In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re



In [3]:
df=pd.read_csv("../raw_data/train.csv")

/tmp/ipykernel_3443/1933274695.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("../raw_data/train.csv")


In [4]:
df.head(5)

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good


In [5]:
df.isnull().sum()

ID                              0
Customer_ID                     0
Month                           0
Name                         9985
Age                             0
SSN                             0
Occupation                      0
Annual_Income                   0
Monthly_Inhand_Salary       15002
Num_Bank_Accounts               0
Num_Credit_Card                 0
Interest_Rate                   0
Num_of_Loan                     0
Type_of_Loan                11408
Delay_from_due_date             0
Num_of_Delayed_Payment       7002
Changed_Credit_Limit            0
Num_Credit_Inquiries         1965
Credit_Mix                      0
Outstanding_Debt                0
Credit_Utilization_Ratio        0
Credit_History_Age           9030
Payment_of_Min_Amount           0
Total_EMI_per_month             0
Amount_invested_monthly      4479
Payment_Behaviour               0
Monthly_Balance              1200
Credit_Score                    0
dtype: int64

In [6]:
#removing any negative or positive 4digit number replacing it to Nan and removing underscore at the end and replacing it with it initial value
def process_age(df, col):
    
    df[col] = df[col].str.replace('_', '')
    df[col] = df[col].str.replace('-', '')
    df[col] = df[col].str.replace(r'\b-\d{3}\b', '',regex=True)
    df[col] = df[col].str.replace(r'\b\d{3}\b', '',regex=True)
    df[col] = df[col].str.replace(r'\b-\d{4}\b', '',regex=True)
    df[col] = df[col].str.replace(r'\b\d{4}\b', '',regex=True)
    df[col] = df[col].replace('', np.nan)
    df[col] = df[col].astype(float)
    
    grouped_df = df.groupby('Customer_ID')
    mode_per_group = grouped_df[col].transform(lambda x: x.mode().iat[0] if len(x.mode()) > 0 else np.nan)
    df[col].fillna(mode_per_group, inplace=True)
    
    return df

In [7]:
clean_age_df = process_age(df,'Age')


In [8]:
clean_age_df['Age'].unique()

array([23., 28., 34., 54., 55., 21., 31., 33., 30., 24., 44., 45., 40.,
       41., 32., 35., 36., 39., 37., 20., 46., 26., 42., 19., 48., 38.,
       43., 22., 16., 18., 15., 27., 25., 14., 17., 47., 53., 56., 29.,
       49., 51., 50., 52., 99., 95.])

In [9]:
def clean_occ(df, col, pattern):
    df[col] = df[col].str.replace(pattern, '')
    df[col] = df[col].replace('', np.nan)
    
    grouped_df = df.groupby('Customer_ID')
    mode_per_group = grouped_df[col].transform(lambda x: x.mode().iat[0] if len(x.mode()) > 0 else np.nan)
    df[col].fillna(mode_per_group, inplace=True)
    
    return df

In [10]:
cleaned_occ_df=clean_occ(clean_age_df,"Occupation","_______")

In [11]:
cleaned_occ_df["Occupation"].unique()

array(['Scientist', 'Teacher', 'Engineer', 'Entrepreneur', 'Developer',
       'Lawyer', 'Media_Manager', 'Doctor', 'Journalist', 'Manager',
       'Accountant', 'Musician', 'Mechanic', 'Writer', 'Architect'],
      dtype=object)

In [12]:
def clean_ai(df, col):
    
    df[col] = df[col].str.replace('_', '')
    df[col] = df[col].replace('', np.nan)    
    grouped_df = df.groupby('Customer_ID')
    mode_per_group = grouped_df[col].transform(lambda x: x.mode().iat[0])
    df[col].fillna(mode_per_group, inplace=True)
    
    return df

In [13]:
clean_ai_df=clean_ai(cleaned_occ_df,"Annual_Income")

In [14]:
clean_ai_df["Annual_Income"].unique()

array(['19114.12', '34847.84', '143162.64', ..., '37188.1', '20002.88',
       '39628.99'], dtype=object)

In [15]:
clean_ai_df.columns

Index(['ID', 'Customer_ID', 'Month', 'Name', 'Age', 'SSN', 'Occupation',
       'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts',
       'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 'Type_of_Loan',
       'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit',
       'Num_Credit_Inquiries', 'Credit_Mix', 'Outstanding_Debt',
       'Credit_Utilization_Ratio', 'Credit_History_Age',
       'Payment_of_Min_Amount', 'Total_EMI_per_month',
       'Amount_invested_monthly', 'Payment_Behaviour', 'Monthly_Balance',
       'Credit_Score'],
      dtype='object')

In [16]:
clean_ai_df["Monthly_Inhand_Salary"].unique()

array([1824.84333333,           nan, 3037.98666667, ..., 3097.00833333,
       1929.90666667, 3359.41583333])

In [17]:
def clean_mis(df, col):
    grouped_df = df.groupby('Customer_ID')
    mode_per_group = grouped_df[col].transform(lambda x: x.mode().iat[0]if len(x.mode()) > 0 else np.nan)
    df[col].fillna(mode_per_group, inplace=True)
    
    return df

In [18]:
clean_mis_df=clean_mis(clean_ai_df,"Monthly_Inhand_Salary")

In [19]:
clean_mis_df.isnull().sum()

ID                              0
Customer_ID                     0
Month                           0
Name                         9985
Age                             0
SSN                             0
Occupation                      0
Annual_Income                   0
Monthly_Inhand_Salary           0
Num_Bank_Accounts               0
Num_Credit_Card                 0
Interest_Rate                   0
Num_of_Loan                     0
Type_of_Loan                11408
Delay_from_due_date             0
Num_of_Delayed_Payment       7002
Changed_Credit_Limit            0
Num_Credit_Inquiries         1965
Credit_Mix                      0
Outstanding_Debt                0
Credit_Utilization_Ratio        0
Credit_History_Age           9030
Payment_of_Min_Amount           0
Total_EMI_per_month             0
Amount_invested_monthly      4479
Payment_Behaviour               0
Monthly_Balance              1200
Credit_Score                    0
dtype: int64

In [20]:
clean_mis_df["Num_Bank_Accounts"].unique()

array([   3,    2,    1,    7,    4,    0,    8,    5,    6,    9,   10,
       1414, 1231,   67,  572, 1488,   91,  528, 1647, 1696, 1338,  649,
        889, 1668,  685,  857,  975, 1496, 1534, 1620,   37, 1388, 1429,
       1332, 1588,  120, 1777, 1096,  803,  494,  744, 1139,  831, 1613,
        741,  121,  665, 1748, 1644,  823, 1356, 1651,   87,  711,  450,
        210, 1671, 1722,  648,  672, 1662, 1495,  510,  666,  429, 1131,
       1434,  980, 1026,  526,  718, 1300, 1312, 1315,  501, 1566,  480,
       1042,  322,  628, 1016,  333, 1265,  791,  427,  563,  905, 1190,
       1036, 1611,   38,  619,  555, 1769,  280,  752, 1201,  812, 1487,
        222,  212, 1019, 1222,  610, 1714, 1308,  525, 1158, 1115, 1616,
        754, 1601,  797, 1577,  521,  703,  933,  959,  809, 1089, 1656,
       1113, 1715,  964, 1533, 1789,  871,  434, 1295,  565, 1724,  603,
        677,  541,  698, 1101,  464, 1539, 1462, 1011, 1257, 1727,   45,
        244, 1266,  897,  535,  484,  331,  675,  8

In [21]:
def clean_nba(df, col):
    df[col] = df[col].apply(lambda x: re.sub(r'\b\d{2}\b', '', str(x)))
    df[col] = df[col].apply(lambda x: re.sub(r'\b\d{3}\b', '', str(x)))
    df[col] = df[col].apply(lambda x: re.sub(r'\b\d{4}\b', '', str(x)))
    df[col] = df[col].replace('', np.nan)
    
    # Remove negative values from the resulting array
    df[col] = df[col].apply(lambda x: x if pd.isnull(x) or int(x) >= 0 else np.nan)
    
    grouped_df = df.groupby('Customer_ID')
    mode_per_group = grouped_df[col].transform(lambda x: x.mode().iat[0] if len(x.mode()) > 0 else np.nan)
    df[col].fillna(mode_per_group, inplace=True)

    return df

In [22]:
clean_nba_df=clean_nba(clean_mis_df,"Num_Bank_Accounts")

In [23]:
clean_nba_df["Num_Bank_Accounts"].unique()

array(['3', '2', '1', '7', '4', '0', '8', '5', '6', '9', nan],
      dtype=object)

In [24]:
clean_nba_df["Num_Credit_Card"].unique()

array([   4, 1385,    5, ...,  955, 1430,  679])

In [25]:
def clean_nbc(df, col):
    df[col] = df[col].apply(lambda x: re.sub(r'\b\d{2}\b', '', str(x)))
    df[col] = df[col].apply(lambda x: re.sub(r'\b\d{3}\b', '', str(x)))
    df[col] = df[col].apply(lambda x: re.sub(r'\b\d{4}\b', '', str(x)))
    df[col] = df[col].replace('', np.nan)
    # df[col] = df[col].astype(float)
    grouped_df = df.groupby('Customer_ID')
    mode_per_group = grouped_df[col].transform(lambda x: x.mode().iat[0]if len(x.mode()) > 0 else np.nan)
    df[col].fillna(mode_per_group, inplace=True)

    return df

In [26]:
clean_nbc_df=clean_nbc(clean_nba_df,'Num_Credit_Card')

In [27]:
clean_nbc_df['Num_Credit_Card'].unique()

array(['4', '5', '1', '7', '6', '8', '3', '9', '2', nan, '0'],
      dtype=object)

In [28]:
clean_nbc_df['Interest_Rate'].unique()

array([   3,    6,    8, ..., 1347,  387, 5729])

In [29]:
def clean_ir(df, col):
    df[col] = df[col].apply(lambda x: re.sub(r'\b\d{3}\b', '', str(x)))
    df[col] = df[col].apply(lambda x: re.sub(r'\b\d{4}\b', '', str(x)))
    df[col] = df[col].replace('', np.nan)
    # df[col] = df[col].astype(float)
    grouped_df = df.groupby('Customer_ID')
    mode_per_group = grouped_df[col].transform(lambda x: x.mode().iat[0]if len(x.mode()) > 0 else np.nan)
    df[col].fillna(mode_per_group, inplace=True)

    return df

In [30]:
clean_ir_df=clean_ir(clean_nbc_df,'Interest_Rate')

In [31]:
clean_ir_df['Interest_Rate'].unique()

array(['3', '6', '8', '4', '5', '15', '7', '12', '20', '1', '14', '32',
       '16', '17', '10', '31', '25', '18', '19', '9', '24', '13', '33',
       '11', '21', '29', '28', '30', '23', '34', '2', '27', '26', '22',
       '73', '76', '94', '74', '67', '87', '75', '59', '89', '95', '91',
       '60', '81', '64', '98', '77', '99', '92'], dtype=object)

In [32]:
clean_ir_df['Num_of_Loan'].unique()

array(['4', '1', '3', '967', '-100', '0', '0_', '2', '3_', '2_', '7', '5',
       '5_', '6', '8', '8_', '9', '9_', '4_', '7_', '1_', '1464', '6_',
       '622', '352', '472', '1017', '945', '146', '563', '341', '444',
       '720', '1485', '49', '737', '1106', '466', '728', '313', '843',
       '597_', '617', '119', '663', '640', '92_', '1019', '501', '1302',
       '39', '716', '848', '931', '1214', '186', '424', '1001', '1110',
       '1152', '457', '1433', '1187', '52', '1480', '1047', '1035',
       '1347_', '33', '193', '699', '329', '1451', '484', '132', '649',
       '995', '545', '684', '1135', '1094', '1204', '654', '58', '348',
       '614', '1363', '323', '1406', '1348', '430', '153', '1461', '905',
       '1312', '1424', '1154', '95', '1353', '1228', '819', '1006', '795',
       '359', '1209', '590', '696', '1185_', '1465', '911', '1181', '70',
       '816', '1369', '143', '1416', '455', '55', '1096', '1474', '420',
       '1131', '904', '89', '1259', '527', '1241', '449', 

In [33]:
def clean_nol(df, col):
    
    df[col] = df[col].str.replace('_', '')
    df[col] = df[col].str.replace('-', '')
    df[col] = df[col].str.replace(r'\b-\d{3}\b', '',regex=True)
    df[col] = df[col].str.replace(r'\b\d{3}\b', '',regex=True)
    df[col] = df[col].str.replace(r'\b-\d{4}\b', '',regex=True)
    df[col] = df[col].str.replace(r'\b\d{4}\b', '',regex=True)
    df[col] = df[col].replace('', np.nan)
    df[col] = df[col].astype(float)
    
    grouped_df = df.groupby('Customer_ID')
    mode_per_group = grouped_df[col].transform(lambda x: x.mode().iat[0] if len(x.mode()) > 0 else np.nan)
    df[col].fillna(mode_per_group, inplace=True)
    
    return df

In [34]:
clean_nol_df=clean_nol(clean_ir_df,"Num_of_Loan")

In [35]:
clean_nol_df['Num_of_Loan'].unique()

array([ 4.,  1.,  3.,  0.,  2.,  7.,  5.,  6.,  8.,  9., 49., 92., 39.,
       52., 33., 58., 95., 70., 55., 89., 23., 31., 32., 50., 29., 87.,
       27., 19., 65., 17., 54., 78., 84., 83., 56., 18., 41.])

In [36]:
clean_nol_df['Type_of_Loan'].unique()



array(['Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan',
       'Credit-Builder Loan', 'Auto Loan, Auto Loan, and Not Specified',
       ..., 'Home Equity Loan, Auto Loan, Auto Loan, and Auto Loan',
       'Payday Loan, Student Loan, Mortgage Loan, and Not Specified',
       'Personal Loan, Auto Loan, Mortgage Loan, Student Loan, and Student Loan'],
      dtype=object)

In [37]:
def clean_tpo(df, col):
    grouped_df = df.groupby('Customer_ID')
    mode_per_group = grouped_df[col].transform(lambda x: x.mode().iat[0] if len(x.mode()) > 0 else np.nan)
    df[col].fillna(mode_per_group, inplace=True)
    
    return df

In [38]:
clean_tpo_df=clean_tpo(clean_nol_df,'Type_of_Loan')

In [39]:
clean_tpo_df['Type_of_Loan'].unique()


array(['Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan',
       'Credit-Builder Loan', 'Auto Loan, Auto Loan, and Not Specified',
       ..., 'Home Equity Loan, Auto Loan, Auto Loan, and Auto Loan',
       'Payday Loan, Student Loan, Mortgage Loan, and Not Specified',
       'Personal Loan, Auto Loan, Mortgage Loan, Student Loan, and Student Loan'],
      dtype=object)

In [40]:
clean_tpo_df.columns


Index(['ID', 'Customer_ID', 'Month', 'Name', 'Age', 'SSN', 'Occupation',
       'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts',
       'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 'Type_of_Loan',
       'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit',
       'Num_Credit_Inquiries', 'Credit_Mix', 'Outstanding_Debt',
       'Credit_Utilization_Ratio', 'Credit_History_Age',
       'Payment_of_Min_Amount', 'Total_EMI_per_month',
       'Amount_invested_monthly', 'Payment_Behaviour', 'Monthly_Balance',
       'Credit_Score'],
      dtype='object')

In [41]:
clean_tpo_df['Num_of_Delayed_Payment'].unique()


array(['7', nan, '4', '8_', '6', '1', '-1', '3_', '0', '8', '5', '3', '9',
       '12', '15', '17', '10', '2', '2_', '11', '14', '20', '22', '13',
       '13_', '14_', '16', '12_', '18', '19', '23', '24', '21', '3318',
       '3083', '22_', '1338', '4_', '26', '11_', '3104', '21_', '25',
       '10_', '183_', '9_', '1106', '834', '19_', '24_', '17_', '23_',
       '2672', '20_', '2008', '-3', '538', '6_', '1_', '16_', '27', '-2',
       '3478', '2420', '15_', '707', '708', '26_', '18_', '3815', '28',
       '5_', '1867', '2250', '1463', '25_', '7_', '4126', '2882', '1941',
       '2655', '2628', '132', '3069', '306', '0_', '3539', '3684', '1823',
       '4128', '1946', '827', '2297', '2566', '904', '182', '929', '3568',
       '2503', '1552', '2812', '1697', '3764', '851', '3905', '923', '88',
       '1668', '3253', '808', '2689', '3858', '642', '3457', '1402',
       '1732', '3154', '847', '3037', '2204', '3103', '1063', '2056',
       '1282', '1841', '2569_', '211', '793', '3484', '4

In [42]:
def clean_ndp(df, col):
    
    df[col] = df[col].str.replace('_', '')
    df[col] = df[col].str.replace('-', '')
    df[col] = df[col].str.replace(r'\b-\d{3}\b', '',regex=True)
    df[col] = df[col].str.replace(r'\b\d{3}\b', '',regex=True)
    df[col] = df[col].str.replace(r'\b-\d{4}\b', '',regex=True)
    df[col] = df[col].str.replace(r'\b\d{4}\b', '',regex=True)
    df[col] = df[col].replace('', np.nan)
    df[col] = df[col].astype(float)
    
    grouped_df = df.groupby('Customer_ID')
    mode_per_group = grouped_df[col].transform(lambda x: x.mode().iat[0] if len(x.mode()) > 0 else np.nan)
    df[col].fillna(mode_per_group, inplace=True)
    
    return df

In [43]:
clean_ndp_df=clean_ndp(clean_tpo_df,'Num_of_Delayed_Payment')

In [44]:
clean_ndp_df['Num_of_Delayed_Payment'].unique()


array([ 7.,  4.,  8.,  6.,  1.,  3.,  0.,  5.,  9., 15., 12., 17., 10.,
        2., 11., 14., 20., 22., 13., 16., 19., 18., 21., 23., 24., 26.,
       25., 27., 28., 88., 84., 72., 47., 52., 49., 98., 80., 46., 94.,
       85., 86.])

In [45]:
clean_ndp_df['Num_Credit_Inquiries'].unique()

array([   4.,    2.,    3., ..., 1361.,  310.,   74.])

In [46]:
def clean_ncq(df, col):
    grouped_df = df.groupby('Customer_ID')
    mode_per_group = grouped_df[col].transform(lambda x: x.mode().iat[0]if len(x.mode()) > 0 else np.nan)
    df[col].fillna(mode_per_group, inplace=True)

    return df

In [47]:
clean_ncq_df=clean_ncq(clean_ndp_df,"Num_Credit_Inquiries")

In [48]:
clean_ncq_df['Credit_Mix'].unique()

array(['_', 'Good', 'Standard', 'Bad'], dtype=object)

In [49]:
def clean_cm(df, col):
    
    df[col] = df[col].str.replace('_', '')
    df[col] = df[col].str.replace('-', '')
    df[col] = df[col].replace('', np.nan)    
    grouped_df = df.groupby('Customer_ID')
    mode_per_group = grouped_df[col].transform(lambda x: x.mode().iat[0] if len(x.mode()) > 0 else np.nan)
    df[col].fillna(mode_per_group, inplace=True)
    
    return df

In [50]:
clean_cm_df=clean_cm(clean_ncq_df,'Credit_Mix')

In [51]:
clean_cm_df['Credit_Mix'].unique()

array(['Good', 'Standard', 'Bad'], dtype=object)

In [52]:
clean_cm_df['Amount_invested_monthly'].unique()





array(['80.41529543900253', '118.28022162236736', '81.699521264648', ...,
       '24.02847744864441', '251.67258219721603', '167.1638651610451'],
      dtype=object)

In [53]:
def clean_aim(df, col):
    
    df[col] = df[col].str.replace('_', '')
    df[col] = df[col].str.replace('-', '')
    df[col] = df[col].replace('', np.nan)    
    grouped_df = df.groupby('Customer_ID')
    mode_per_group = grouped_df[col].transform(lambda x: x.mode().iat[0] if len(x.mode()) > 0 else np.nan)
    df[col].fillna(mode_per_group, inplace=True)
    
    return df

In [54]:
clean_aim_df=clean_aim(clean_cm_df,'Amount_invested_monthly')

In [55]:
clean_aim_df['Amount_invested_monthly'].unique()


array(['80.41529543900253', '118.28022162236736', '81.699521264648', ...,
       '24.02847744864441', '251.67258219721603', '167.1638651610451'],
      dtype=object)

In [56]:
clean_aim_df['Payment_Behaviour'].unique()


array(['High_spent_Small_value_payments',
       'Low_spent_Large_value_payments',
       'Low_spent_Medium_value_payments',
       'Low_spent_Small_value_payments',
       'High_spent_Medium_value_payments', '!@9#%8',
       'High_spent_Large_value_payments'], dtype=object)

In [57]:
def clean_pb(df, col):
    
    df[col] = df[col].str.replace('!@9#%8', '')
    df[col] = df[col].replace('', np.nan)    
    grouped_df = df.groupby('Customer_ID')
    mode_per_group = grouped_df[col].transform(lambda x: x.mode().iat[0] if len(x.mode()) > 0 else np.nan)
    df[col].fillna(mode_per_group, inplace=True)
    
    return df

In [58]:
clean_pb_df=clean_pb(clean_aim_df,'Payment_Behaviour')

In [59]:
clean_pb_df['Payment_Behaviour'].unique()


array(['High_spent_Small_value_payments',
       'Low_spent_Large_value_payments',
       'Low_spent_Medium_value_payments',
       'Low_spent_Small_value_payments',
       'High_spent_Medium_value_payments',
       'High_spent_Large_value_payments'], dtype=object)

In [60]:
clean_pb_df['Monthly_Balance'].unique()



array(['312.49408867943663', '284.62916249607184', '331.2098628537912',
       ..., 516.8090832742814, 319.1649785257098, 393.6736955618808],
      dtype=object)

In [61]:
def clean_mb(df, col):
    
    df[col] = df[col].str.replace('__-333333333333333333333333333__', '')
    df[col] = df[col].replace('', np.nan)    
    grouped_df = df.groupby('Customer_ID')
    mode_per_group = grouped_df[col].transform(lambda x: x.mode().iat[0] if len(x.mode()) > 0 else np.nan)
    df[col].fillna(mode_per_group, inplace=True)
    
    return df

In [62]:
clean_pb_df['Monthly_Balance'].unique()



array(['312.49408867943663', '284.62916249607184', '331.2098628537912',
       ..., 516.8090832742814, 319.1649785257098, 393.6736955618808],
      dtype=object)

In [63]:
clean_mb_df=clean_mb(clean_pb_df,'Monthly_Balance')


In [64]:
clean_mb_df['Monthly_Balance'].unique()


array(['312.49408867943663', '284.62916249607184', '331.2098628537912',
       ..., '343.79399959878214', '214.61493361875802', nan], dtype=object)

In [65]:
clean_mb_df['Credit_History_Age'].unique()


array(['22 Years and 1 Months', nan, '22 Years and 3 Months',
       '22 Years and 4 Months', '22 Years and 5 Months',
       '22 Years and 6 Months', '22 Years and 7 Months',
       '26 Years and 7 Months', '26 Years and 8 Months',
       '26 Years and 9 Months', '26 Years and 10 Months',
       '26 Years and 11 Months', '27 Years and 0 Months',
       '27 Years and 1 Months', '27 Years and 2 Months',
       '17 Years and 9 Months', '17 Years and 10 Months',
       '17 Years and 11 Months', '18 Years and 1 Months',
       '18 Years and 2 Months', '18 Years and 3 Months',
       '18 Years and 4 Months', '17 Years and 3 Months',
       '17 Years and 4 Months', '17 Years and 5 Months',
       '17 Years and 6 Months', '17 Years and 7 Months',
       '17 Years and 8 Months', '30 Years and 8 Months',
       '30 Years and 9 Months', '30 Years and 10 Months',
       '30 Years and 11 Months', '31 Years and 0 Months',
       '31 Years and 1 Months', '31 Years and 2 Months',
       '31 Years and

In [66]:
def convert_to_months(duration):
    if isinstance(duration, str):
        years, months = duration.split(' Years and ')
        months = months.replace(' Months', '')
        total_months = int(years) * 12 + int(months)
        return total_months
    else:
        return duration

In [67]:
clean_mb_df['Credit_History_Age'] = clean_mb_df['Credit_History_Age'].apply(convert_to_months)


In [68]:
def clean_cha(df, col):
    grouped_df = df.groupby('Customer_ID')
    mode_per_group = grouped_df[col].transform(lambda x: x.mode().iat[0] if len(x.mode()) > 0 else np.nan)
    df[col].fillna(mode_per_group, inplace=True)
    
    return df

In [69]:
clean_cha_df=clean_cha(clean_mb_df,'Credit_History_Age')

In [70]:
clean_cha_df['Credit_History_Age'].isnull().sum()

0

In [71]:
month_mapping = {
    'January': 1,
    'February': 2,
    'March': 3,
    'April': 4,
    'May': 5,
    'June': 6,
    'July': 7,
    'August': 8
}

clean_cha_df['Month'] = clean_cha_df['Month'].map(month_mapping)

In [72]:
clean_cha_df['Month']

0        1
1        2
2        3
3        4
4        5
        ..
99995    4
99996    5
99997    6
99998    7
99999    8
Name: Month, Length: 100000, dtype: int64

In [73]:
#converting numerical features

columns_to_convert = ['Annual_Income', 'Num_Bank_Accounts', 'Num_Credit_Card', 'Interest_Rate', 'Changed_Credit_Limit', 'Outstanding_Debt', 'Amount_invested_monthly', 'Monthly_Balance']

clean_cha_df[columns_to_convert] = clean_cha_df[columns_to_convert].apply(pd.to_numeric, errors='coerce')

In [74]:
clean_cha_df.dtypes

ID                           object
Customer_ID                  object
Month                         int64
Name                         object
Age                         float64
SSN                          object
Occupation                   object
Annual_Income               float64
Monthly_Inhand_Salary       float64
Num_Bank_Accounts           float64
Num_Credit_Card             float64
Interest_Rate                 int64
Num_of_Loan                 float64
Type_of_Loan                 object
Delay_from_due_date           int64
Num_of_Delayed_Payment      float64
Changed_Credit_Limit        float64
Num_Credit_Inquiries        float64
Credit_Mix                   object
Outstanding_Debt            float64
Credit_Utilization_Ratio    float64
Credit_History_Age          float64
Payment_of_Min_Amount        object
Total_EMI_per_month         float64
Amount_invested_monthly     float64
Payment_Behaviour            object
Monthly_Balance             float64
Credit_Score                

In [75]:
clean_cha_df.isnull().sum()

ID                              0
Customer_ID                     0
Month                           0
Name                         9985
Age                             0
SSN                             0
Occupation                      0
Annual_Income                   0
Monthly_Inhand_Salary           0
Num_Bank_Accounts            5280
Num_Credit_Card              4928
Interest_Rate                   0
Num_of_Loan                     0
Type_of_Loan                11408
Delay_from_due_date             0
Num_of_Delayed_Payment          0
Changed_Credit_Limit         2091
Num_Credit_Inquiries            0
Credit_Mix                      0
Outstanding_Debt             1009
Credit_Utilization_Ratio        0
Credit_History_Age              0
Payment_of_Min_Amount           0
Total_EMI_per_month             0
Amount_invested_monthly         0
Payment_Behaviour               0
Monthly_Balance              1696
Credit_Score                    0
dtype: int64

In [76]:
clean_cha_df.dropna(inplace=True)

In [77]:
clean_cha_df.tail()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
98299,0x255f9,CUS_0x9d41,4,Rothackerz,38.0,979-38-2855,Lawyer,41015.55,3152.9625,0.0,...,Good,599.4,31.936092,302.0,NM,25.671257,387.541439,Low_spent_Small_value_payments,192.083554,Standard
98300,0x255fa,CUS_0x9d41,5,Rothackerz,38.0,979-38-2855,Lawyer,41015.55,3152.9625,0.0,...,Good,599.4,26.268474,303.0,No,25.671257,76.057218,High_spent_Large_value_payments,453.567775,Standard
98301,0x255fb,CUS_0x9d41,6,Rothackerz,38.0,979-38-2855,Lawyer,41015.55,3152.9625,0.0,...,Good,599.4,28.754729,299.0,No,25.671257,91.198586,High_spent_Medium_value_payments,448.426407,Standard
98302,0x255fc,CUS_0x9d41,7,Rothackerz,38.0,979-38-2855,Lawyer,41015.55,3152.9625,0.0,...,Good,599.4,38.514974,305.0,No,3466.000000,205.830993,High_spent_Small_value_payments,343.794000,Standard
98303,0x255fd,CUS_0x9d41,8,Rothackerz,38.0,979-38-2855,Lawyer,41015.55,3152.9625,0.0,...,Good,599.4,31.356267,306.0,No,25.671257,10000.000000,Low_spent_Small_value_payments,214.614934,Standard


In [78]:
clean_cha_df.isnull().sum()

ID                          0
Customer_ID                 0
Month                       0
Name                        0
Age                         0
SSN                         0
Occupation                  0
Annual_Income               0
Monthly_Inhand_Salary       0
Num_Bank_Accounts           0
Num_Credit_Card             0
Interest_Rate               0
Num_of_Loan                 0
Type_of_Loan                0
Delay_from_due_date         0
Num_of_Delayed_Payment      0
Changed_Credit_Limit        0
Num_Credit_Inquiries        0
Credit_Mix                  0
Outstanding_Debt            0
Credit_Utilization_Ratio    0
Credit_History_Age          0
Payment_of_Min_Amount       0
Total_EMI_per_month         0
Amount_invested_monthly     0
Payment_Behaviour           0
Monthly_Balance             0
Credit_Score                0
dtype: int64

In [79]:
clean_cha_df.shape

(67965, 28)

In [80]:
columns_to_drop = ['ID', 'Customer_ID', 'Name','SSN']
final_df = clean_cha_df.drop(columns_to_drop, axis=1)

In [89]:
final_df1=final_df.sample(10000)

X=final_df1.drop(columns=['Credit_Score'])
y=final_df1['Credit_Score']

In [90]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)


In [91]:

one_hot_encode_cols = ['Occupation', 'Type_of_Loan', 'Payment_Behaviour']
ordinal_encode_cols = ['Payment_of_Min_Amount','Credit_Mix']
min_max_scale_cols = ['Age', 'Num_Bank_Accounts', 'Num_Credit_Card', 'Num_Credit_Inquiries',
                      'Num_of_Delayed_Payment']
standard_scale_cols = ['Credit_Utilization_Ratio', 'Changed_Credit_Limit']
robust_scale_cols = ['Annual_Income', 'Monthly_Inhand_Salary', 'Interest_Rate', 'Num_of_Loan',
                     'Delay_from_due_date', 'Outstanding_Debt', 'Total_EMI_per_month',
                     'Amount_invested_monthly', 'Monthly_Balance']

In [92]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, MinMaxScaler, StandardScaler, RobustScaler, LabelEncoder

column_transformations = [
    ('one_hot_encode', OneHotEncoder( sparse=False,handle_unknown='ignore'), one_hot_encode_cols),
    ('ordinal_encode', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), ordinal_encode_cols)
]

scaling_transformer = [('min_max_scale', MinMaxScaler(), min_max_scale_cols),
                       ('standard_scale', StandardScaler(), standard_scale_cols),
                       ('robust_scale', RobustScaler(), robust_scale_cols)]

preprocessor = ColumnTransformer(transformers=column_transformations +
                                               scaling_transformer)

pipeline = make_pipeline(preprocessor)

X_train_preprocessed = pipeline.fit_transform(X_train)
X_test_preprocessed = pipeline.transform(X_test)




In [93]:
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)

In [94]:
from tpot import TPOTClassifier
from sklearn.metrics import accuracy_score



tpot = TPOTClassifier(generations=5, population_size=20, verbosity=2)
tpot.fit(X_train_preprocessed, y_train_encoded)


# Print the best pipeline found by TPOT
print(tpot.fitted_pipeline_)

# Evaluate the best pipeline on the test data
accuracy = tpot.score(X_test_preprocessed, y_test_encoded)
print("Accuracy:", accuracy)



Optimization Progress:   0%|          | 0/120 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.73325

Generation 2 - Current best internal CV score: 0.73325

Generation 3 - Current best internal CV score: 0.73325

Generation 4 - Current best internal CV score: 0.73325

Generation 5 - Current best internal CV score: 0.73325

Best pipeline: RandomForestClassifier(input_matrix, bootstrap=False, criterion=gini, max_features=0.1, min_samples_leaf=2, min_samples_split=5, n_estimators=100)
Pipeline(steps=[('randomforestclassifier',
                 RandomForestClassifier(bootstrap=False, max_features=0.1,
                                        min_samples_leaf=2,
                                        min_samples_split=5))])
Accuracy: 0.7385
